# Ising Model Monte Carlo Simulation

The following describes the Ising model, its Monte Carlo simulation using **Metropolis** and **Wolff cluster algorithms**.

---

## 1. Ising Hamiltonian

The classical Ising Hamiltonian:

$$
\mathcal{H} = - \sum_{\langle i,j \rangle} s_i s_j - h \sum_i s_i
$$

where:

- $s_i \in \{-1, +1\}$ is the spin at site $i$.
- $\langle i,j \rangle$ denotes nearest-neighbor pairs.
- $h$ is the external magnetic field.
- Periodic boundary conditions are assumed.
- The inverse temperature is:
  
$$
\beta = \frac{1}{k_B T} \quad (\text{with } k_B = 1)
$$

**Physical interpretation:**  
- Spins aligned with neighbors minimize energy.  
- Temperature controls thermal fluctuations.

---

## 2. Lattice and State Representation (`Ising` class)

Each lattice site $i$ is represented by its coordinates (tuple), and neighbors are stored in a **neighbor table**:

$$
\text{neighbours\_table}[k][i] = j
$$

- Different lattices (1D, square, triangular, honeycomb) correspond to different **coordination numbers** $z$.  
- Geometry affects critical temperature $T_c$ and universality class.

---

## 3. Single-Spin Energy Change (`energy_singleflip`)

For flipping a spin $s_i \rightarrow -s_i$, the energy change is:

$$
\Delta E = 2 s_i \left( \sum_{j \in \text{nn}(i)} s_j + h \right)
$$

- This computes **energy difference locally**, avoiding recalculation of total energy.
- Large $\Delta E > 0$ → flip is energetically costly, less likely at low $T$.

---

## 4. Metropolis Update (`metropolis_step`)

The Metropolis acceptance probability:

$$
P_{\text{accept}} =
\begin{cases}
1, & \Delta E \le 0 \\
e^{-\beta \Delta E}, & \Delta E > 0
\end{cases}
$$

**Physical interpretation:**  
- Ensures **detailed balance** and samples the **Boltzmann distribution**:

$$
P(\{s\}) \propto e^{-\beta \mathcal{H}}
$$

- Represents thermal fluctuations of individual spins.

---

## 5. Metropolis Sweep (`metropolis_sweep`)

- Attempts $N = \prod_d L_d$ single-spin updates per sweep.
- Roughly corresponds to one **Monte Carlo “time step”**.  
- Ensures all spins can potentially flip once per sweep.

---

## 6. Wolff Cluster Algorithm (`wolff`)

Near the **critical temperature**, large domains of aligned spins form. Single-spin flips become inefficient (**critical slowing down**). The Wolff algorithm flips **entire clusters of aligned spins** to accelerate sampling.

---

### 6.2 Cluster Formation

1. Pick a random seed spin $s_i$.
2. Build a cluster recursively:
   - For each neighbor $j$ of a spin in the cluster:
     - If $s_j = s_i$, add to the cluster with probability:

$$
p_{\text{add}} = 1 - e^{-2 \beta}
$$

3. Flip all spins in the cluster simultaneously:

$$
s_k \to -s_k, \quad \forall k \in \text{cluster}
$$

**Physical interpretation:**  
- Collective flips mimic large correlated fluctuations.
- Reduces **autocorrelation times** near $T_c$.

---

### 6.3 Recursive Implementation (`create_cluster`)

- Recursively checks neighbors, marking sites in `ising.cluster`.
- Flips spins as the cluster grows.
- Ensures **ergodicity** and preserves **detailed balance**.

---

### 6.4 Iterative Version

- Python recursion limit can be hit for large clusters.
- Iterative stack-based implementations avoid this.

---

## 7. Magnetization (`magnetization`)

Total magnetization:

$$
M = \left| \sum_i s_i \right|
$$

**Physical interpretation:**  
- Measures long-range order:
  - $M > 0$: ferromagnetic
  - $M \approx 0$: paramagnetic

Absolute value removes symmetry between $+M$ and $-M$.

---

## 8. Total Energy (`energy`)

Total energy:

$$
E = - \sum_{\langle i,j \rangle} s_i s_j - h \sum_i s_i
$$

- Implementation avoids double counting by dividing by 2.
- Used to compute **energy per spin** and **heat capacity**.

---

## 9. Observables Accumulation (`update_observables`)

- Average energy and magnetization:

$$
\langle E \rangle = \frac{1}{N_{\text{meas}}} \sum_{n=1}^{N_{\text{meas}}} E_n
\quad , \quad
\langle M \rangle = \frac{1}{N_{\text{meas}}} \sum_{n=1}^{N_{\text{meas}}} M_n
$$

- Measurements taken **after equilibration**.

---

## 10. Monte Carlo Run (`run`)

Algorithm:

1. Thermalization phase: first $N$ sweeps
2. Measurement phase: next $N$ sweeps
3. Each sweep:
   - Metropolis update
   - Wolff cluster update

- Hybrid algorithm combines **local** and **global moves**.
- Ensures fast convergence and accurate sampling.

---

## 11. Thermodynamic Limits

Per-spin observables:

$$
e = \frac{\langle E \rangle}{N_{\text{sites}}}, \quad
m = \frac{\langle M \rangle}{N_{\text{sites}}}
$$

- As $N_{\text{sites}} \to \infty$, these approach thermodynamic values.

